In [3]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


%load_ext autoreload
%autoreload 2

from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def calc_margin(ab1, ab2):
    dmag = (ab1 - ab2)
    flux_ratio = 10**(0.4*dmag)
    margin = flux_ratio - 1.0
    return margin


def flux_ratio(ab1, ab2):
    dmag = (ab1 - ab2)
    flux_ratio = 10**(0.4*dmag)
    return flux_ratio

In [5]:

# Band1
# 5-sigma limiting magnitude in 3 stacked frames, CBE

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1.0]: #, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('QE loss: {}'.format(qe_loss))
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()


duet1_cbe_limit = bbmag1



B
QE loss: 1.0
Band1 900.0 s 5-σ magnitude limit: 22.43368761357837 mag(AB)
Rates:  0.1396049959121671 ph / s 0.01753365098332382 ph / s
41.88149877365013 ph 5.260095294997146 ph



In [6]:

# Band2
# 5-sigma limiting magnitude in 3 stacked frames,  CBE

duet = config.Telescope()
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()
filter_type = 'B'
for qe_loss in [1.0]:#, 0.8, 0.77, 0.7, 0.5, 0.4]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('QE loss: {}'.format(qe_loss))
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()



duet2_cbe_limit = bbmag2



B
QE loss: 1.0
Band2 900.0 s 5-σ magnitude limit: 21.76198025121982 mag(AB)
Rates:  0.2429509268401836 ph / s 0.17922799624269026 ph / s
72.88527805205509 ph 53.768398872807076 ph



In [7]:

# Band1
# 5-sigma limiting magnitude in 3 stacked frames for 2.3x QE drop

duet = config.Telescope()

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1/2.3]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print()



print('Margin: {}'.format(calc_margin(duet1_cbe_limit.value, bbmag1.value)))
print('Flux Ratio: {}'.format(flux_ratio(duet1_cbe_limit.value, bbmag1.value)))




B
Band1 900.0 s 5-σ magnitude limit: 21.59368761357824 mag(AB)

Margin: 1.1677041048199568
Flux Ratio: 2.167704104819957


In [9]:

# Band2
# 5-sigma limiting magnitude in 3 stacked frames for 2.3x QE drop

duet = config.Telescope()

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.01
print()

filter_type = 'B'
for qe_loss in [1/2.3]:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2 * qe_loss

    for nframes in [3]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate * qe_loss

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=12e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))
        print()



print('Margin: {}'.format(calc_margin(duet2_cbe_limit.value, bbmag2.value)))
print('Flux Ratio: {}'.format(flux_ratio(duet2_cbe_limit.value, bbmag2.value)))








B
Band1 900.0 s 5-σ magnitude limit: 20.809882231071313 mag(AB)

Margin: 1.403472780404536
Flux Ratio: 2.403472780404536
